In [1]:
import rasterio
from rasterio.mask import mask
import geopandas as gpd
import os

# Function to clip a raster file with a shapefile
def clip_raster_with_shapefile(raster_path, shapefile_path, output_path):
    """
    Clip a raster file using a shapefile and save the new raster.

    Parameters:
        raster_path (str): Path to the input raster file.
        shapefile_path (str): Path to the input shapefile.
        output_path (str): Path to save the clipped raster.
    """
    try:
        # Read the shapefile
        shapefile = gpd.read_file(shapefile_path)

        # Ensure the shapefile and raster CRS match
        with rasterio.open(raster_path) as src:
            raster_crs = src.crs

        if shapefile.crs != raster_crs:
            shapefile = shapefile.to_crs(raster_crs)

        # Extract the geometry from the shapefile
        shapes = [feature['geometry'] for feature in shapefile.iterfeatures()]

        # Open the raster file
        with rasterio.open(raster_path) as src:
            # Clip the raster using the shapefile geometry
            out_image, out_transform = mask(src, shapes, crop=True)
            out_meta = src.meta.copy()

        # Update metadata for the clipped raster
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })

        # Save the clipped raster
        with rasterio.open(output_path, "w", **out_meta) as dest:
            dest.write(out_image)

        print(f"Clipped raster saved to: {output_path}")

    except Exception as e:
        print(f"Error during clipping: {e}")

# Example usage
# Provide paths to the raster file, shapefile, and output file
raster_path = r"C:\Users\91956\Downloads\TN_FCD\FCDmerged.tif"
shapefile_path = r"C:\Users\91956\Downloads\Green_elements_400.gpkg"
output_path = r"C:\Users\91956\Downloads\fcd_treecover.tif"


clip_raster_with_shapefile(raster_path, shapefile_path, output_path)


Clipped raster saved to: C:\Users\91956\Downloads\fcd_treecover.tif
